In [4]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal

from qubx import QubxLogConfig
from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet
from qubx.trackers.sizers import FixedSizer

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Strat for signals testing

In [10]:
class StrategyForTracking(IStrategy):
    timeframe: str = "1Min"
    fast_period = 5
    slow_period = 12

    def on_fit(self, ctx: "StrategyContext", fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f" -> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        signals = []
        for i in ctx.instruments:
            # logger.info(f"\t{i.symbol} : {ctx.broker_provider.get_quote(i.symbol)}")
            ohlc = ctx.ohlc(i, self.timeframe)
            fast = sma(ohlc.close, self.fast_period)
            slow = sma(ohlc.close, self.slow_period)
            pos = ctx.positions[i.symbol].quantity

            if pos <= 0:
                if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                    # ctx.trade(i, abs(pos) + i.min_size * 10)
                    signals.append(i.signal(+1))

            if pos >= 0:
                if (fast[0] < slow[0]) and (fast[1] > slow[1]):
                    # ctx.trade(i, -pos - i.min_size * 10)
                    signals.append(i.signal(-1))

        return signals

    def ohlcs(self, timeframe: str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def tracker(self, ctx: StrategyContext) -> PositionsTracker:
        return PositionsTracker(FixedSizer(0.01))

In [3]:
# r = MultiQdbConnector('xlydian-data')

2024-07-29 08:19:35.657 [ 🐞 ] Connected to QuestDB at xlydian-data:8812


In [ ]:
r = CsvStorageDataReader("../tests/data/csv")

QubxLogConfig.set_log_level("DEBUG")

rep = simulate({
    
        "As Strategy": StrategyForTracking(
            timeframe="5Min", fast_period=5, slow_period=15),
    },
    r, 10000, ["BINANCE.UM:BTCUSDT"], 
    dict(type="ohlc", timeframe="1Min", nback=0),
    "5Min -1Sec",
    "vip0_usdt", 
    "2024-01-01", "2024-01-02", 
)

In [ ]:
tearsheet(rep)

In [5]:
import numpy as np

def _how_much_can_be_closed(self, position: float, to_remain: float) -> float:
    d = np.sign(position)
    qty_to_close = position
    if to_remain != 0 and position != 0 and np.sign(to_remain) == d:
        qty_to_close = max(position - to_remain, 0) if d > 0 else min(position - to_remain, 0)
    return qty_to_close

In [18]:
_how_much_can_be_closed(None, 1000, 10)

990